In [ ]:
# Text feature extraction

In [ ]:
# NB Only for text_referall_SA_SD_s47 - copy for df_outcome1
# named df_rq1 to save re-writing everything but only text_referall_SA_SD_s47 post-anonymisation

In [ ]:
# Add to system path so we can find all the packages
import sys
sys.path
sys.path.append('C:\\Program Files\\Python37\\Lib\\site-packages')
sys.path.append('C:\Program Files\Python37')

# Load user-written functions

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

#import text_functions

In [ ]:
# Checking number of documents
import pickle
import os

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]

filename = open("Created\\rq1data_text_for_anonymisation.pkl", "rb")
rq1data_text_for_anonymisation = pickle.load(filename)
print(rq1data_text_for_anonymisation.shape)

rq1_text_cols = (['Contact and Referral Form_text',
       'Child Social Work Assessment for Review Child Protection Conference_text_prev',
       'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
       'Child Social Work Assessment_text_prev',
       'Contact and Referral Form_text_prev'])

rq1_no_docs = 0
for col in rq1_text_cols:
    print(col)
    non_empty = sum(rq1data_text_for_anonymisation[col] != '')
    print(non_empty)
    rq1_no_docs += non_empty
    
print("Number of documents for rq1: ", rq1_no_docs)

# Rq2 
filename = open("Created\\rq2data_text_for_anonymisation.pkl", "rb")
rq2data_text_for_anonymisation = pickle.load(filename)
print(rq2data_text_for_anonymisation.shape)
rq2_text_cols = (['Child Social Work Assessment for Review Child Protection Conference_text',
       'Child Social Work Assessment to Initial Child Protection Conference_text',
       'Child Social Work Assessment_text', 'Contact and Referral Form_text',
       'Child Social Work Assessment for Review Child Protection Conference_text_prev',
       'Child Social Work Assessment to Initial Child Protection Conference_text_prev',
       'Child Social Work Assessment_text_prev',
       'Contact and Referral Form_text_prev'])

rq2_no_docs = 0
for col in rq2_text_cols:
    print(col)
    non_empty = sum(rq2data_text_for_anonymisation[col] != '')
    print(non_empty)
    rq2_no_docs += non_empty
    
print("Number of documents for rq2: ", rq2_no_docs)

In [ ]:
import os
import pickle
os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created\\text_rq1_names_matched.pkl", "rb")
text = pickle.load(filename)
print(text.shape)

### Bring in emotion and concreteness lexicons

In [ ]:
# Call emotion lexicon
# emolex_words is a dataframe of words and 0/1 as to whether they are associated with particular emotion
import pandas as pd
filepath = ('''Other\\Lexicons\\NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt''')
emolex_df = pd.read_csv(filepath, skiprows=28,
                        names=["word", "emotion", "association"],
                        sep='\t')
emolex_words = emolex_df.pivot(index='word',
                               columns='emotion',
                               values='association').reset_index()


In [ ]:
# Concreteness
# concreteness_df is a dataframe with the word and "Conc.M" is the mean of a concreteness score given by MTurks
filepath = ('''Other\\Lexicons\\Concreteness_ratings_Brysbaert_et_al_BRM.txt''')
concreteness_df = pd.read_csv(filepath,sep='\t')
concreteness_df.head()

In [ ]:
#!pip install -U textblob
#!python -m textblob.download_corporafrom textblob 

In [ ]:
# Identify name and text columns
text_columns = list(set(text.columns) - set(['PSID', 'ReferralDatetime', 'ReferralDatetime_previous']))
print(text_columns)

In [ ]:
from text_functions import text_feature_creation
import numpy as np
import pickle

# Create features relating to polarity, subjectivity, emotions and concreteness using text_feature_creation function
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']

df_emotions_list = []
for column in text_columns[3:]:
    print(column)
    # Create text features
    df_emotions = text_feature_creation(text, column, emotions)
    df_emotions_list.append(df_emotions)
    
with open("Created\\df_emotions_list.pkl", "wb") as handle:
    pickle.dump(df_emotions_list, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [ ]:
# Combine the IDs with the features columns
df_text_features = pd.concat(df_emotions_list, axis = 1)
#print(df_text_features.head())

with open("Created\\df_text_features.pkl", "wb") as handle:
    pickle.dump(df_text_features, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Create vulnerabilities

In [ ]:
# Social worker stop words
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")
social_work_stop_words = (["children's social care", 
                           "case worker", "social worker", "duty worker", "foster carer", "sw",
                        "contact", "referral", "single assessment", "assessment", 
                           "initial child protection conference", "child in need", "child protection", 
                       "strategy discussion", "section 47", "section 17", "section 20" , "looked after",
                           "no further action", "enquiries", "recommendation", 
                           "SA", "S47", "S17", "S20", "CP", "CIN", "NFA", "LAC", "CLA", "CAMHS", "child"])

# Adding social work stop words
for word in social_work_stop_words:
    STOP_WORDS.add(word)

for i, word in enumerate(STOP_WORDS):
    lexeme = nlp.vocab[i]
    lexeme.is_stop = True

In [ ]:
import numpy as np
import spacy

from text_functions import RecogniseTagPhrases, vulnerabilities_phrases_dict

# Add vulnerabilities to recognise
nlp = spacy.load("en_core_web_sm")
vulnerabilities_phrases = [y for x in vulnerabilities_phrases_dict.values() for y in x]
vulnerabilities_merger = RecogniseTagPhrases(nlp, vulnerabilities_phrases, "is_vulnerability")
nlp.add_pipe(vulnerabilities_merger, last=True)  # Add component to the pipeline

# Other:  "step mother", "step father" - how to classify?

text_columns = (['text_rq2_names_matched_Child Social Work Assessment to Initial Child Protection Conference_text'])
rq = 'rq2'

save_every = 100

text_vulnerabilities = pd.DataFrame(0, index=text.index, columns = vulnerabilities_phrases_dict.keys())
text_list_of_strings = pd.DataFrame(None, index=text.index, columns = text_columns, dtype = 'object')
text_tokenised = pd.DataFrame(None, index=text.index, columns = text_columns, dtype = 'object')
# Pre-process text data (lemmatise, remove punctuation, remove stop words)
for col in text_columns:
    print("Column: ", col)
    text_list_of_strings[col] = ''
    text_tokenised[col] = ''
    already_done_indices = []
    for idx in text[col].index:
        print(idx)
        if int(idx) not in already_done_indices:
            string = text.loc[idx, col]
            # If string is NaN
            if string != string:
                print(string)
                #print(type(string[0]))
                # Replacing NaN with empty strings
                text_list_of_strings.at[idx, col] = ''
                text_tokenised.at[idx, col] = ''
            else:
                doc = nlp(string) 
                # Make dummy if vulnerability appears in any of the columns
                vulnerabilities = set([token.text for token in doc if token._.is_vulnerability is True])
                if vulnerabilities:
                    vulnerabilities_grouped = set([key for key, value in vulnerabilities_phrases_dict.items() for v in value if v in vulnerabilities])
                    #print("Vulnerabilities:", vulnerabilities)
                    print("Vulnerabilities grouped:", vulnerabilities_grouped)
                    for vulnerability in vulnerabilities_grouped:
                        text_vulnerabilities.at[idx, vulnerability] = 1
                # Lemmatise and remove punctuation
                alphas = [token for token in doc if token.is_alpha]
                #print("Alphas: ", alphas)
                # Remove stop words including social worker stop words
                not_stop = [token for token in alphas if token.is_stop is False]
                #print("Stop: ", [token for token in alphas if token.is_stop is True])
                lemmas = [token.lemma_ for token in not_stop]
                #print("Lemmas", lemmas)
                text_list_of_strings.at[idx, col] = ' '.join(lemmas)# sklearn tfidf vectoriser wants list of strings
                text_tokenised.at[idx, col] = lemmas # gensim lda wants list of list of strings
                # If already processed, use processed data
                repeated_text_indices = text.index[np.where(text[col] ==  text.loc[idx,col])]
                print("Repeated text indices: ", repeated_text_indices)
                already_done_indices.extend(list(repeated_text_indices))
                if repeated_text_indices != []:
                    # String
                    text_list_of_strings.loc[repeated_text_indices, col] = ' '.join(lemmas)
                    # List of tokens needs more complex version
                    text_tokenised_repeated = pd.concat([pd.DataFrame(text_tokenised.loc[idx,]).T]*len(repeated_text_indices))
                    text_tokenised_repeated.set_index(repeated_text_indices, inplace = True)
                    text_tokenised.update(text_tokenised_repeated)
                    # Dataframe needs more complex version
                    if vulnerabilities:
                        text_vulnerabilities_repeated = pd.concat([pd.DataFrame(text_vulnerabilities.loc[idx,]).T]*len(repeated_text_indices))
                        text_vulnerabilities_repeated.set_index(repeated_text_indices, inplace = True)
                        text_vulnerabilities.update(text_vulnerabilities_repeated)
        if (idx % save_every == 0) & (idx != 0):
            # text_vulnerabilities column names describe the vulnerabilities not the document => save all
            with open("Created/Preprocessed Text/text_{}_vulnerabilities_{}_{}_{}.pkl".format(rq, col, idx - save_every, idx - 1), "wb") as handle:
                pickle.dump(text_vulnerabilities.loc[idx-save_every: idx - 1,], handle, protocol = pickle.HIGHEST_PROTOCOL)   
            with open("Created/Preprocessed Text/text_{}_list_of_strings_{}_{}_{}.pkl".format(rq, col, idx - save_every, idx - 1), "wb") as handle:
                pickle.dump(text_list_of_strings.loc[idx-save_every: idx - 1, col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
            with open("Created/Preprocessed Text/text_{}_tokenised_{}_{}_{}.pkl".format(rq, col, idx - save_every, idx - 1), "wb") as handle:
                pickle.dump(text_tokenised.loc[idx-save_every: idx - 1, col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
        elif (idx == text.shape[0] - 1):
            with open("Created/Preprocessed Text/text_{}_vulnerabilities_{}_{}_{}.pkl".format(rq, col, idx - idx % save_every - 1, idx), "wb") as handle:
                pickle.dump(text_vulnerabilities.loc[idx - idx % save_every: idx,], handle, protocol = pickle.HIGHEST_PROTOCOL)   
            with open("Created/Preprocessed Text/text_{}_list_of_strings_{}_{}_{}.pkl".format(rq, col, idx - idx % save_every - 1, idx), "wb") as handle:
                pickle.dump(text_list_of_strings.loc[idx - idx % save_every: idx, col], handle, protocol = pickle.HIGHEST_PROTOCOL)   
            with open("Created/Preprocessed Text/text_{}_tokenised_{}_{}_{}.pkl".format(rq, col, idx - idx % save_every - 1, idx), "wb") as handle:
                    pickle.dump(text_tokenised.loc[idx - idx % save_every: idx, col], handle, protocol = pickle.HIGHEST_PROTOCOL)    


# Count number of vulnerabilities
text_vulnerabilities["Number of vulnerabilities"] = text_vulnerabilities[list(vulnerabilities_phrases_dict.keys())].sum(axis=1)


In [ ]:
### Test Preprocessed

# Knit together files
import glob
import os
import pickle
import pandas as pd
import re

# assert 1==2
# Run for 'list_of_strings', 'tokenised', 'vulnerabilities'
# Note different text_cols for rq2

rq = 'rq1'
file_type = 'vulnerabilities'

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename_list = [file for file in glob.glob("Created\\Preprocessed Text\\text_{}_{}_*.pkl".format(rq, file_type))]


# Call in all files in the anonymisation folder
file_dict = {}
for file in filename_list:
    try:
        filename = open(file, "rb")
        f = pickle.load(filename)
        file_n = re.sub('.pkl', '', file)
        file_n = re.sub("Created\\\\Preprocessed Text\\\\text_{}_{}_".format(rq, file_type), '', file_n)
        print(file_n)
        file_dict[file_n] = f
    except(EOFError):
        continue

col_wo_identifier_dict = {}
for text_col in text_cols:
    text_col_df = [df for df in file_dict.keys() if text_col in df]
    if 'prev' in text_col:
        text_col_df = [df for df in text_col_df if 'prev' in df]
    else:
        text_col_df = [df for df in text_col_df if 'prev' not in df]
    print(text_col_df)
    text_col_df_values = [v for k, v in file_dict.items() if k in text_col_df]
    print(len(text_col_df_values))
    col_wo_identifier_dict[text_col] = pd.concat(text_col_df_values, axis = 0)
    col_wo_identifier_dict[text_col].sort_index(inplace = True)
    
from functools import reduce

# Deduplicate by index (in case any rows are repeated in different files)
for col, df in col_wo_identifier_dict.items():
    print(col)
    print(df.shape)
    col_wo_identifier_dict[col] = df.loc[~df.index.duplicated(keep = 'first')]
    print(col_wo_identifier_dict[col].shape)
    if file_type == 'vulnerabilities':
        col_wo_identifier_dict[col] = col_wo_identifier_dict[col].add_prefix(col + '_')
    else:
        col_wo_identifier_dict[col].rename(col, inplace = True)

# Merge together on index
df_wo_identifier_together = reduce(lambda left, right: pd.merge(left,right,left_index = True, right_index = True, how = 'outer'), col_wo_identifier_dict.values())
df_wo_identifier_together = df_wo_identifier_together.loc[~df_wo_identifier_together.index.duplicated(keep = 'first')]



In [ ]:
# Bring in the original text dataset for anonymisation so that we can merge in the IDs
# Reset index so the text lines up df_wo_identifier_together
# The problem was a combination of the indices starting at 7221, 7222, 0, 1, 2 etc and missing data

import os
import pickle

os.chdir("C:\\Users\\[username]\\Downloads\\Updated Text Data") # insert [username]
filename = open("Created\\{}data_text_for_anonymisation.pkl".format(rq), "rb")
original_text = pickle.load(filename)
print(original_text.index)
original_text.reset_index(drop = True, inplace = True) # to line up with df_wo_identifier_together
print(original_text.index)

In [ ]:
# Merge back in IDs
print(original_text.shape)
print(original_text.columns)
final_text = pd.merge(original_text[['PSID', 'ReferralDatetime', 'ReferralDatetime_previous']], df_wo_identifier_together, left_index = True, right_index = True, how = 'left')
print(final_text.shape)
print(final_text.columns)

# Replace missing with empty string

if file_type != 'vulnerabilities':
    print(final_text.isna().sum()) 
    final_text[text_cols] = final_text[text_cols].fillna('')
    print(final_text.isna().sum())

# Save knitted data all together 
with open("Created/text_{}_{}.pkl".format(rq, file_type), "wb") as handle:
    pickle.dump(final_text, handle, protocol = pickle.HIGHEST_PROTOCOL)